In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, datasets, layers
from keras.datasets import mnist
from matplotlib import pyplot as plt
import numpy as np
import random
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import talib as tb
import time
import os
# import pywhatkit
import matplotlib.pyplot as plt
# from warnings import simplefilt

2023-12-31 17:49:36.466489: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-31 17:49:36.510018: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-31 17:49:36.510484: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 17:49:37.413079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Training data is the data of the previous day<br>
i.e. we are considering trading only at the opening and closing of the market

In [2]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

In [3]:
n50_symbols=np.array(['NIFTY', 'HINDALCO', 'MARUTI', 'NESTLEIND', 'ONGC', 'TATAMOTORS', 'ITC', 'SUNPHARMA', 'BHARTIARTL', 'CIPLA', 'DIVISLAB', 'JSWSTEEL', 'HINDUNILVR', 'TITAN', 'NTPC', 'TATASTEEL', 'HEROMOTOCO', 'HDFCLIFE', 'ULTRACEMCO', 'SBILIFE', 'TCS', 'BAJFINANCE', 'BRITANNIA', 'AXISBANK', 'TECHM', 'COALINDIA', 'LT', 'DRREDDY', 'M_M', 'APOLLOHOSP', 'ADANIPORTS', 'BAJAJFINSV', 'GRASIM', 'TATACONSUM', 'ICICIBANK', 'HDFCBANK', 'EICHERMOT', 'BPCL', 'ADANIENT', 'UPL', 'POWERGRID', 'HDFCBANK', 'INFY', 'WIPRO', 'RELIANCE', 'INDUSINDBK', 'ASIANPAINT', 'HCLTECH', 'KOTAKBANK', 'BAJAJ_AUTO', 'SBIN'])

folder_name = 'Daily_historical_data'

X_train=[]
y_train=[]
y_train_pos=[]
gain=[]


for i in range(51):
    file_name = f'{n50_symbols[i]}.csv'
    file_path = os.path.join(folder_name, file_name)
    df=pd.read_csv(file_path)
    df['datetime']=pd.to_datetime(df['datetime'])
    df['year'] = df['datetime'].dt.year
    # df['month'] = df['datetime'].dt.month
    # df['day_of_week'] = df['datetime'].dt.dayofweek
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute

    ###############################Indicators#######################
 
    #ema of last 5 candle changes
    df['ema200']=tb.EMA(df['close'],200)
    #aroon
    df["aroon"]=tb.AROONOSC(df['high'],df['low'],timeperiod=14)
    # #macd
    # df["macd"]=tb.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]
    #obv
    df["obv_last"]=tb.OBV(df['close'],df['open'])
    #sar
    df["sar_now"]=tb.SAR(df['high'],df['low'],0.02,0.2)
    #adx
    df["adx"]=tb.ADX(df['high'],df['low'],df['close'])
    #bbands
    df['bbands_up']=tb.BBANDS(df['close'],5,2,2,0)[0]
    df['bbands_mid']=tb.BBANDS(df['close'],5,2,2,0)[1]
    df['bbands_down']=tb.BBANDS(df['close'],5,2,2,0)[2]
    #rsi
    df["rsi"]=tb.RSI(df['close'])
    #PPO
    # df["ppo"]=tb.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)

    df=df.iloc[201:].reset_index(drop=True)
    # print(df)

    start_ind=[]
    for j in range(len(df.index)):
        if df['hour'][j]==9 and df['minute'][j]==15: # Getting the indices where the trading day starts
            start_ind.append(j)
    end_ind=[] # end of the day indices
    for j in range(1,len(start_ind)): #starting from 1 because we dont want an end without the start of a day which is the case for start_of_day[0]-1
        end_ind.append(start_ind[j]-1)

    # print(start_ind)
    # print(end_ind)    

    if len(start_ind)>len(end_ind):
        start_ind=start_ind[:-1]

    # print(start_ind)
    # print(end_ind)
    j=0
    while j<len(start_ind):
        if end_ind[j]-start_ind[j]!=74:
            end_ind.remove(end_ind[j])
            start_ind.remove(start_ind[j])
            continue

        # X from 9:25AM to 3:15pm 
        # y from 3:15 to 9:45am the next day
        df_new=df.iloc[start_ind[j]+2:end_ind[j]-2].drop(['symbol','datetime','year','hour','minute'],axis=1)

        ###################         Imputer          #####################
        df_new = pd.DataFrame(imputer.fit_transform(df_new))

        if len(df_new.columns)==len(df.columns)-5:
            X_train.append(df_new.values.flatten())
        else:
            end_ind.remove(end_ind[j])
            start_ind.remove(start_ind[j])
            
        j+=1
    
    print(i)

    for j in end_ind:
        gain_inst=(df['close'][j+7]-df['close'][j-2])*100/df['close'][j-2] # gain at the particular instant
        gain.append(gain_inst)  # i+2 indicates the next day at 9:20(as we need to let the price settle) and i-2 at time 3:15 as we need to trade before the day ends and we cant trade ecaxtly at the end of the day
        if gain_inst>=0.1:
            y_train_pos.append(1)
        elif gain_inst<-0.1:
            y_train_pos.append(-1)
        else:
            y_train_pos.append(0)

y_train=gain

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [4]:
from sklearn.preprocessing import MinMaxScaler

# Normalizing the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle

#shuffled data
X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=42)

rf_regressor=RandomForestRegressor(n_estimators=60,random_state=42)

rf_regressor.fit(X_train_shuffled,y_train_shuffled)

RandomForestRegressor(n_estimators=60, random_state=42)

In [6]:
y_predicted=rf_regressor.predict(X_train)

y_pred_pos=[]
for y in y_predicted:
    if y>0.1:
        y_pred_pos.append(1)
    elif y<-0.1:
        y_pred_pos.append(-1)
    else:
        y_pred_pos.append(0)        

# correct_pred_train=0
# for x,y in zip(y_predicted,y_train):
#     if x==y:
#         correct_pred_train+=1

# correct_pred_train

# Run at 3:15pm
 

In [7]:
tv = TvDatafeed()

x=1

while x==1:

    # df50_init=[]
    X_pred=[]
    y_pred=[]
    y_pos=[]
    price_arr=[]


    for i in range(51):
        data_5_min = tv.get_hist(symbol=f'{n50_symbols[i]}',exchange='NSE',interval=Interval.in_5_minute,n_bars=500)
        df=pd.DataFrame(data_5_min)
        df=df.reset_index()
        # df50_init.append(df)
        # df['datetime']=pd.to_datetime(df['datetime'])
        df['year'] = df['datetime'].dt.year
        # df['month'] = df['datetime'].dt.month
        # df['day_of_week'] = df['datetime'].dt.dayofweek
        df['hour'] = df['datetime'].dt.hour
        df['minute'] = df['datetime'].dt.minute

            ###############################Indicators#######################
    
        #ema of last 5 candle changes
        df['ema200']=tb.EMA(df['close'],200)
        #aroon
        df["aroon"]=tb.AROONOSC(df['high'],df['low'],timeperiod=14)
        # #macd
        # df["macd"]=tb.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]
        #obv
        df["obv_last"]=tb.OBV(df['close'],df['open'])
        #sar
        df["sar_now"]=tb.SAR(df['high'],df['low'],0.02,0.2)
        #adx
        df["adx"]=tb.ADX(df['high'],df['low'],df['close'])
        #bbands
        df['bbands_up']=tb.BBANDS(df['close'],5,2,2,0)[0]
        df['bbands_mid']=tb.BBANDS(df['close'],5,2,2,0)[1]
        df['bbands_down']=tb.BBANDS(df['close'],5,2,2,0)[2]
        #rsi
        df["rsi"]=tb.RSI(df['close'])
        #PPO
        # df["ppo"]=tb.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)

        df=df.iloc[201:].reset_index(drop=True)

        # #print(df)
        # X_end_arr=[]
        # for j in range(len(df.index)):
        #     if df['hour'][j]==15 and df['minute'][j]==15:
        #         X_end_arr.append(j)

        # print(df.iloc[X_end_ind-70:X_end_ind+1])

        df_for_pred=df.iloc[-70:].drop(['symbol','datetime','year','hour','minute'],axis=1)
        #print(df_for_pred)
        X_pred.append(df_for_pred.values.flatten())

        # Price of stock at that instant
        price_arr.append(df['close'][len(df.index)-1])
    
    X_pred_norm=scaler.transform(X_pred)
    y_pred=rf_regressor.predict(X_pred_norm)
    #print(y_pred)

    for gain_p in y_pred:
        if gain_p>0.1:
            y_pos.append(1)
        elif gain_p<-0.1:
            y_pos.append(-1)
        else:
            y_pos.append(0)        
    
    df_pred=pd.DataFrame()
    df_pred['symbol']=n50_symbols
    df_pred['price']=price_arr
    df_pred['gain%']=y_pred
    df_pred['abs gain%']=abs(y_pred)
    df_pred['pos']=y_pos
    df_pred= df_pred.sort_values(by='abs gain%',ascending=False).reset_index(drop=True)

    print(df_pred)
    x+=1

    #time.sleep(300)     

you are using nologin method, data you access may be limited


        symbol     price     gain%  abs gain%  pos
0       GRASIM   2139.80  0.571823   0.571823    1
1     HINDALCO    614.25  0.521078   0.521078    1
2     JSWSTEEL    878.50  0.448004   0.448004    1
3     DIVISLAB   3908.20  0.438149   0.438149    1
4     AXISBANK   1100.10  0.370473   0.370473    1
5    ICICIBANK    998.40  0.323442   0.323442    1
6         INFY   1542.50  0.316201   0.316201    1
7         NTPC    310.50  0.297053   0.297053    1
8        WIPRO    470.25  0.295726   0.295726    1
9      SBILIFE   1432.10  0.283709   0.283709    1
10   COALINDIA    376.20  0.280658   0.280658    1
11  HEROMOTOCO   4144.95  0.268557   0.268557    1
12   BRITANNIA   5340.80  0.268317   0.268317    1
13   POWERGRID    237.00  0.260740   0.260740    1
14   TATASTEEL    139.30  0.256916   0.256916    1
15         UPL    587.65  0.247953   0.247953    1
16        SBIN    641.90  0.246188   0.246188    1
17  APOLLOHOSP   5708.00  0.208451   0.208451    1
18        BPCL    451.00  0.205

In [8]:
tv = TvDatafeed()

# wanted_day_ind=12

df50=[]

for i in range(51):
    data_5_min = tv.get_hist(symbol=f'{n50_symbols[i]}',exchange='NSE',interval=Interval.in_5_minute,n_bars=2000)
    df=pd.DataFrame(data_5_min)
    df=df.reset_index()
    # df50_init.append(df)
    # df['datetime']=pd.to_datetime(df['datetime'])
    df['year'] = df['datetime'].dt.year
    # df['month'] = df['datetime'].dt.month
    # df['day_of_week'] = df['datetime'].dt.dayofweek
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute
    df50.append(df)


profit_m1=[]
profit_m2=[]

for wanted_day_ind in range(2,13):

    X_pred=[]
    y_pred=[]
    y_pos=[]
    price_arr=[]

    gain_test=[]
    change_arr=[]
    act_pos=[]

    for df in df50:
        #print(df)

        #####Indicators####
        df['ema200']=tb.EMA(df['close'],200)
        #aroon
        df["aroon"]=tb.AROONOSC(df['high'],df['low'],timeperiod=14)
        # #macd
        # df["macd"]=tb.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)[0]
        #obv
        df["obv_last"]=tb.OBV(df['close'],df['open'])
        #sar
        df["sar_now"]=tb.SAR(df['high'],df['low'],0.02,0.2)
        #adx
        df["adx"]=tb.ADX(df['high'],df['low'],df['close'])
        #bbands
        df['bbands_up']=tb.BBANDS(df['close'],5,2,2,0)[0]
        df['bbands_mid']=tb.BBANDS(df['close'],5,2,2,0)[1]
        df['bbands_down']=tb.BBANDS(df['close'],5,2,2,0)[2]
        #rsi
        df["rsi"]=tb.RSI(df['close'])
        # #PPO
        # df["ppo"]=tb.PPO(df['close'], fastperiod=12, slowperiod=26, matype=0)
        

        df=df[201:].reset_index(drop=True)

        X_end_arr=[]
        for j in range(len(df.index)):
            if df['hour'][j]==15 and df['minute'][j]==15:
                X_end_arr.append(j)




        df_for_pred=df.iloc[X_end_arr[-wanted_day_ind]-70:X_end_arr[-wanted_day_ind]].drop(['symbol','datetime','year','hour','minute'],axis=1)
        #print(df_for_pred)
        
        df_for_pred = pd.DataFrame(imputer.fit_transform(df_for_pred), columns=df_for_pred.columns)
        
        X_pred.append(df_for_pred.values.flatten())

        # Price of stock at that instant
        price_arr.append(df['close'][X_end_arr[-wanted_day_ind]])

        # ind=end_arr[-1] ####################################################################
        g=(df['close'][X_end_arr[-wanted_day_ind]+9]-df['close'][X_end_arr[-wanted_day_ind]])*100/df['close'][X_end_arr[-wanted_day_ind]]
        gain_test.append(g)
        change=df['close'][X_end_arr[-wanted_day_ind]+9]-df['close'][X_end_arr[-wanted_day_ind]]
        change_arr.append(change)

    X_pred_norm=scaler.transform(X_pred)
    y_pred=rf_regressor.predict(X_pred_norm)
    #print(y_pred)

    for gain_p in y_pred:
        if gain_p>0.1:
            y_pos.append(1)
        elif gain_p<-0.1:
            y_pos.append(-1)
        else:
            y_pos.append(0)   
                
    for gain_p in gain_test:
        if gain_p>0.1:
            act_pos.append(1)
        elif gain_p<-0.1:
            act_pos.append(-1)
        else:
            act_pos.append(0) 

    df_test=pd.DataFrame()
    df_test['symbol']=n50_symbols
    df_test['price']=price_arr
    df_test['pred_gain%']=y_pred
    df_test['act_gain%']=gain_test
    df_test['abs pred gain%']=abs(y_pred)
    df_test['abs act gain%']=abs(df_test['act_gain%'])
    df_test['pred pos']=y_pos
    df_test['actual pos']=act_pos
    df_test['profit']=change_arr
    df_test= df_test.sort_values(by='abs pred gain%',ascending=False).reset_index(drop=True)

    # print(df_test)

    ############## Checking Profits##################

    cost=0
    profit1=0

    balance=10000

    df_top10=df_test.iloc[0:20]

    for i in range(10):
        symb=df_test['symbol'][i]

        if cost+df_test['price'][i]<=balance:
            cost+=df_test['price'][i]
            # print(df_test['symbol'][i])
            profit1+=df_test.loc[i,'pred pos']*df_test.loc[i,'profit']

    # print(f"balance1: {balance}")
    # print(f"Method1 profit: {profit1-0.0005*cost}")
    profit_m1.append(profit1-0.0005*cost)

    ######### Method2 ############

    profit=0
    cost=0  
    balance=10000

    quantity=np.zeros(10)
    df_top10=df_test.iloc[0:10]

    for i in range(10):
        symb=df_test['symbol'][i]
        quantity[i]=(balance/10)//df_test['price'][i]
        cost+=quantity[i]*df_test['price'][i]
        profit+=quantity[i]*df_test.loc[i,'pred pos']*df_test.loc[i,'profit']

    #Using the remaining balance 

    for i in range(10):
        symb=df_test['symbol'][i]
        if cost+df_test['price'][i]<=balance:
            quantity[i]+=1
            cost+=df_test['price'][i]
            profit+=df_test.loc[i,'pred pos']*df_test.loc[i,'profit']

    balance-=cost

    df_top10['quantity']=quantity

    # print(f"balance2: {balance}")
    # print(f"Method2 profit: {profit-0.0005*cost}")
    profit_m2.append(profit-0.0005*cost)


print(profit_m1)
print(profit_m2)
   

/tmp/ipykernel_17444/2634154167.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top10['quantity']=quantity
/tmp/ipykernel_17444/2634154167.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top10['quantity']=quantity
/tmp/ipykernel_17444/2634154167.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

[-16.39070000000001, 28.71967499999959, 106.83107500000023, 78.82862500000019, 85.17800000000001, -9.011024999999272, 67.37182500000036, -17.266524999999874, -38.52474999999942, 64.12949999999988, 94.43930000000043]
[-26.069000000000056, -4.111150000000636, 99.73757500000009, 45.456524999999985, 123.64317499999994, 9.2116500000005, 71.95840000000024, 4.736849999999931, -41.17269999999935, 97.26602499999989, 81.82995000000048]


/tmp/ipykernel_17444/2634154167.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top10['quantity']=quantity


In [9]:
print(np.array(profit_m1).sum())
print(np.array(profit_m2).sum())

444.3050000000021
462.487300000001
